In [106]:
import os
import numpy as np
import glob
import csv
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats, optimize
from pandas import DataFrame, Series
import seaborn as sns
import random as rd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import scipy.stats
import patsy
from scipy.optimize import minimize
from sklearn import linear_model

##Code for analysis of fMRI experiment

In [2]:
%matplotlib inline

In [182]:
#Read in data
data_dir = '/Users/ianballard/Dropbox/fd/'
all_rts = pd.read_csv(data_dir + 'all_rts.csv', index_col =0)
exp_order = pd.read_csv(data_dir + 'exp_order.csv', index_col =0)
num_runs = 3

In [213]:
#process the RTs a bit (remove trial types 1, 4, and 10 (see below) and mean center)
ser_rts = all_rts[all_rts['exp']=='ser']
ser_rts = ser_rts[ser_rts['order'] != 10] #ITI
ser_rts = ser_rts[ser_rts['order'] != 1] #A
ser_rts = ser_rts[ser_rts['order'] != 4] #A

order_dict = {'c_plus':6, 'b_plus':2, 'b_minus':3, 'c_minus':5} #coding for trial order vector


In [222]:
def perform_RL(trial_order,rew_trial,alpha):
    V = {'b_plus':[0], 'b_minus' : [0], 'c_plus' : [0],'c_minus':[0]}
    index = {'b_plus':[], 'b_minus' : [], 'c_plus' : [],'c_minus':[]}
    count = 0
    for n,cond in enumerate(trial_order):
        trial_type = None
        for key in order_dict:
            if order_dict[key] == cond:
                trial_type = key
        
        if trial_type is not None:
            rew = rew_trial[n]
            delta = rew - V[trial_type][-1]
            new_V = V[trial_type][-1] + alpha * delta
            V[trial_type].append(new_V)
            index[trial_type].append(count)
            count += 1   
    return V, index


In [237]:
##function for building dataframe of relevant data for each subject
def build_df(sub):
    predictors = {'V':[],'run':[], 'cond':[], 'trial_index':[],'rt':[]}
    for i in range(1,num_runs+1):

        ##perform RL for the experimental condition
        rew = np.array(exp_order[exp_order['run']==i]['rew'])
        tt = np.array(exp_order[exp_order['run']==i]['trial_order'])
        V, index = perform_RL(tt,rew,alpha)

        #get rt data for this subject and run
        rt_data = ser_rts[(ser_rts['sub']==sub) & (ser_rts['run']==i) ]
        
        ##update predictors dict
        for key in V.keys():
            V[key] = V[key][:-1] #last entry is for subsequent trial that doesnt exist
            predictors['V'].extend(V[key])
            predictors['cond'].extend([key]*len(V[key]))
            predictors['run'].extend(['run' + str(i)]*len(V[key]))
            predictors['trial_index'].extend(index[key])
            predictors['rt'].extend(rt_data[rt_data['order']==order_dict[key]]['rt'].values)

    predictors = pd.DataFrame(predictors)
    predictors = predictors.sort(['run','trial_index']) #get predictors in proper order

    #mean center RT
    predictors['rt'] = predictors['rt'] - predictors['rt'].mean()

    #Z-score trial index for each row
    for i in range(1,num_runs+1):
        run = 'run' + str(i)
        predictors.loc[predictors['run']==run,'trial_index'] =  predictors.loc[predictors['run']==run,'trial_index'] - \
        predictors.loc[predictors['run']==run,'trial_index'].mean()
        predictors.loc[predictors['run']==run,'trial_index'] =  predictors.loc[predictors['run']==run,'trial_index'] / \
        predictors.loc[predictors['run']==run,'trial_index'].std()
    
    return predictors

In [240]:
##returns loss for linear regression
def regress(params):
    
    alpha = params[0] #learning rate
    beta = params[1:]
    
    ##perform RL for the experimental condition
    for i in range(1,num_runs+1):
        rew = np.array(exp_order[exp_order['run']==i]['rew'])
        tt = np.array(exp_order[exp_order['run']==i]['trial_order'])
        V, index = perform_RL(tt,rew,alpha)
        for key in V.keys():
            V[key] = V[key][:-1] #last entry is for subsequent trial that doesnt exist
            predictors.loc[(predictors['run'] == 'run' + str(i)) & (predictors['cond'] == key),'V'] = V[key]

    #build RL matrixes
    yd,Xd = patsy.dmatrices("rt ~ 1+trial_index+V+run",predictors,NA_action='drop')
    X = np.asarray(Xd)
    y=np.array(map(float,np.asarray(yd)))

    #compute prediction and loss
    y_hat = np.dot(X,beta)
    loss = np.linalg.norm(y - y_hat)
    return loss

In [241]:
predictors = build_df(112)
params = np.transpose(np.zeros(X.shape[1]+1)) #initialize parameters to 0
res = minimize(regress, params, method='BFGS',options={'disp': True})

Optimization terminated successfully.
         Current function value: 276.558365
         Iterations: 20
         Function evaluations: 168
         Gradient evaluations: 21


In [242]:
res

   status: 0
  success: True
     njev: 21
     nfev: 168
 hess_inv: array([[  1.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,  13.68201753, -14.47335064, -13.81820957,
          0.22098521,   0.        ],
       [  0.        , -14.47335064,  28.77724693,  13.98171067,
         -0.70184257,   0.        ],
       [  0.        , -13.81820957,  13.98171067,  25.19194916,
         -0.47551846,   0.        ],
       [  0.        ,   0.22098521,  -0.70184257,  -0.47551846,
          4.47410093,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ]])
      fun: 276.5583650839056
        x: array([  0.        ,   0.59423704, -13.1730896 ,  11.85249137,
        -3.8587171 ,   0.        ])
  message: 'Optimization terminated successfully.'
      jac: array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
        -3.81469727e-06,   0.00000000e+00,   0.00000000e+00])

In [110]:
#double check that linear regression code works (given a fixed alpha)
clf = linear_model.LinearRegression(fit_intercept=False)
clf.fit(X, y)
clf.coef_

array([ -3.52411454, -13.38725855,  11.71133923,  -9.9083914 ,  24.31352455])